### Install dependencies

In [ ]:
# training_app.py

# Dependencies installation

# Run these commands first:
!pip install datasets transformers evaluate accelerate -U
!pip install huggingface_hub
!pip install streamlit mlflow peft
!pip install -q gradio
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install pyngrok
!pip install torch>=2.0.0
!npm install localtunnel
!pip install mlflow
!pip install pyngrok
!pip install mlflow[extras]
!pip install nltk rouge-score seaborn

In [ ]:
!pip install databricks-cli

### Import the libraries

In [ ]:
import warnings
import mlflow
warnings.filterwarnings("ignore")
print(mlflow.__version__)

In [ ]:
import os
import warnings
import mlflow
import torch
import streamlit as st
from pathlib import Path
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainerCallback
)
from torch.utils.data import Dataset

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model
)
from mlflow.tracking import MlflowClient
from accelerate import Accelerator
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [ ]:
!huggingface-cli login

### Setup MLflow

In [ ]:
warnings.filterwarnings("ignore")
print(f"MLflow version: {mlflow.__version__}")

# Set up the MLflow tracking URI
local_registry = "sqlite:///mlruns.db"
print(f"Running local model registry={local_registry}")
mlflow.set_tracking_uri(local_registry)

MODEL_NAME = "Documently_Model"

### Setup the model

#### Model and tokenizer & Dataset prep

In [ ]:
def setup_model_and_tokenizer(model_name="google/gemma-2b"):
    """Setup the model and tokenizer with MLflow tracking."""
    with mlflow.start_run(run_name="MODEL_SETUP") as run:
        compute_dtype = torch.float16
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True
        )

        # Log configuration parameters
        mlflow.log_params({
            "model_name": model_name,
            "compute_dtype": str(compute_dtype),
            "quantization": "4-bit",
            "quant_type": "nf4"
        })

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            model_max_length=512,
            padding_side="right",
            use_fast=True,
        )

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=compute_dtype,
        )

        model = prepare_model_for_kbit_training(
            model,
            use_gradient_checkpointing=True,
        )

        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )

        # Log LoRA configuration
        mlflow.log_params({
            "lora_r": lora_config.r,
            "lora_alpha": lora_config.lora_alpha,
            "lora_dropout": lora_config.lora_dropout
        })

        model = get_peft_model(model, lora_config)

        # Log model summary
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        mlflow.log_metrics({
            "trainable_parameters": trainable_params,
            "total_parameters": total_params,
            "trainable_percentage": (trainable_params / total_params) * 100
        })

        return model, tokenizer, run.info.run_id

def load_and_prepare_dataset(num_examples=2000):
    """Load and prepare the dataset with MLflow tracking."""
    with mlflow.start_run(run_name="DATA_PREPARATION") as run:
        dataset = load_dataset("code_search_net", "python",trust_remote_code=True)
        df = dataset["train"].to_pandas()
        df = df[df['func_documentation_string'].str.len() > 0].reset_index(drop=True)
        df = df.head(num_examples)

        prompts = []
        completions = []

        for _, row in df.iterrows():
            prompt = f"Write documentation for this Python function:\n{row['func_code_string']}"
            completion = f"\nDocumentation:\n{row['func_documentation_string']}\n"
            prompts.append(prompt)
            completions.append(completion)

        # Enhanced dataset analysis and visualization
        import matplotlib.pyplot as plt
        import seaborn as sns
        from collections import Counter
        import numpy as np

        # Calculate and plot length distributions
        prompt_lengths = [len(p) for p in prompts]
        completion_lengths = [len(c) for c in completions]

        # Create length distribution plot
        plt.figure(figsize=(10, 6))
        plt.hist([prompt_lengths, completion_lengths], label=['Prompts', 'Completions'], bins=30, alpha=0.7)
        plt.xlabel('Length (characters)')
        plt.ylabel('Frequency')
        plt.title('Distribution of Prompt and Completion Lengths')
        plt.legend()
        mlflow.log_figure(plt.gcf(), "length_distribution.png")
        plt.close()

        # Calculate and log additional dataset statistics
        mlflow.log_metrics({
            "num_examples": len(prompts),
            "max_length": max(len(p) + len(c) for p, c in zip(prompts, completions)),
            "avg_length": sum(len(p) + len(c) for p, c in zip(prompts, completions)) / len(prompts),
            "min_prompt_length": min(prompt_lengths),
            "max_prompt_length": max(prompt_lengths),
            "avg_prompt_length": np.mean(prompt_lengths),
            "min_completion_length": min(completion_lengths),
            "max_completion_length": max(completion_lengths),
            "avg_completion_length": np.mean(completion_lengths),
            "std_prompt_length": np.std(prompt_lengths),
            "std_completion_length": np.std(completion_lengths)
        })

        return {
            "prompts": prompts,
            "completions": completions
        }, run.info.run_id

def prepare_training_data(dataset_info, tokenizer):
    """Prepare the training data by tokenizing and creating a PyTorch Dataset."""
    combined_texts = [p + c for p, c in zip(dataset_info["prompts"], dataset_info["completions"])]

    tokenized_data = tokenizer(
        combined_texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    class DocumentationDataset(Dataset):
        def __init__(self, tokenized_data):
            self.input_ids = tokenized_data['input_ids']
            self.attention_mask = tokenized_data['attention_mask']

        def __len__(self):
            return len(self.input_ids)

        def __getitem__(self, idx):
            return {
                'input_ids': self.input_ids[idx],
                'attention_mask': self.attention_mask[idx],
                'labels': self.input_ids[idx].clone()
            }

    return DocumentationDataset(tokenized_data)



class MetricsCallback(TrainerCallback):
    """Custom callback to log additional metrics during training."""
    def __init__(self, tokenizer, eval_dataset, eval_steps=100, run_id=None):
        self.tokenizer = tokenizer
        self.eval_dataset = eval_dataset
        self.eval_steps = eval_steps
        self.losses = []
        self.bleu_scores = []
        self.rouge_scores = []
        self.step_numbers = []
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.run_id = run_id  # Store the run_id

    def on_step_end(self, args, state, control, model=None, **kwargs):
        if state.global_step % self.eval_steps == 0:
            # Ensure we're in the correct MLflow run context
            with mlflow.start_run(run_id=self.run_id, nested=True) as run:
                model.eval()
                sample_size = min(10, len(self.eval_dataset))
                indices = np.random.choice(len(self.eval_dataset), sample_size, replace=False)

                bleu_scores = []
                rouge1_scores = []
                rouge2_scores = []
                rougeL_scores = []

                for idx in indices:
                    inputs = self.eval_dataset[idx]
                    input_ids = inputs['input_ids'].unsqueeze(0).to(model.device)

                    with torch.no_grad():
                        outputs = model.generate(
                            input_ids,
                            max_new_tokens=128,  # Add this line to control the number of new tokens generated
                            num_return_sequences=1,
                            pad_token_id=self.tokenizer.pad_token_id
                        )

                    predicted_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                    target_text = self.tokenizer.decode(inputs['labels'], skip_special_tokens=True)

                    # Calculate BLEU score
                    smoothing = SmoothingFunction().method1
                    bleu_score = sentence_bleu(
                        [target_text.split()],
                        predicted_text.split(),
                        smoothing_function=smoothing
                    )
                    bleu_scores.append(bleu_score)

                    # Calculate ROUGE scores
                    rouge_scores = self.scorer.score(target_text, predicted_text)
                    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
                    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
                    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

                # Get current loss
                current_loss = state.log_history[-1].get('loss', 0) if state.log_history else 0

                # Log metrics
                metrics = {
                    'step': state.global_step,
                    'loss': current_loss,
                    'bleu_score': np.mean(bleu_scores),
                    'rouge1_score': np.mean(rouge1_scores),
                    'rouge2_score': np.mean(rouge2_scores),
                    'rougeL_score': np.mean(rougeL_scores)
                }

                # Log metrics to MLflow
                mlflow.log_metrics(metrics, step=state.global_step)
                print(metrics)

                # Store metrics for plotting
                self.losses.append(current_loss)
                self.bleu_scores.append(np.mean(bleu_scores))
                self.step_numbers.append(state.global_step)

                # Create and log plots
                if len(self.step_numbers) > 1:
                    # Loss plot
                    plt.figure(figsize=(10, 6))
                    plt.plot(self.step_numbers, self.losses)
                    plt.xlabel('Training Steps')
                    plt.ylabel('Loss')
                    plt.title('Training Loss Over Time')
                    mlflow.log_figure(plt.gcf(), f"loss_plot_step_{state.global_step}.png")
                    plt.close()

                    # BLEU score plot
                    plt.figure(figsize=(10, 6))
                    plt.plot(self.step_numbers, self.bleu_scores)
                    plt.xlabel('Training Steps')
                    plt.ylabel('BLEU Score')
                    plt.title('BLEU Score Over Time')
                    mlflow.log_figure(plt.gcf(), f"bleu_plot_step_{state.global_step}.png")
                    plt.close()

                model.train()

def train_model(model, tokenizer, dataset, training_args):
    """Train the model with MLflow tracking."""
    with mlflow.start_run(run_name="MODEL_TRAINING") as run:
        # Log training parameters
        mlflow.log_params(training_args)

        # Initialize custom metrics callback with the current run_id
        metrics_callback = MetricsCallback(
            tokenizer=tokenizer,
            eval_dataset=dataset,
            eval_steps=100,
            run_id=run.info.run_id  # Pass the run_id to the callback
        )

        trainer = Trainer(
            model=model,
            args=TrainingArguments(**training_args),
            train_dataset=dataset,
            data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
            callbacks=[metrics_callback]
        )

        # Train the model and log metrics
        train_result = trainer.train()
        mlflow.log_metrics(train_result.metrics)

        # Save the model's state dictionary
        model_path = "model"
        os.makedirs(model_path, exist_ok=True)
        torch.save(trainer.model.state_dict(), os.path.join(model_path, "model_state_dict.pth"))

        # Log the model directory as an artifact
        mlflow.log_artifacts(model_path, artifact_path="model")

        # Register the model with MLflow
        mlflow.register_model(
            model_uri=f"runs:/{run.info.run_id}/model",
            name=MODEL_NAME
        )

        return trainer, run.info.run_id

### Training the model

In [ ]:
import os
import shutil
import mlflow
from mlflow.tracking import MlflowClient

def setup_mlflow():
    # Define the base directory for MLflow
    base_dir = "/content/mlflow"

    # Clean up any existing mlruns.db file that might be causing conflicts
    if os.path.exists("/content/mlruns.db"):
        os.remove("/content/mlruns.db")

    # Create a fresh directory for MLflow
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    os.makedirs(base_dir, exist_ok=True)

    # Set up MLflow to use the local directory
    mlflow.set_tracking_uri(f"file:{base_dir}")

    # Create the .trash directory if it doesn't exist
    trash_dir = os.path.join(base_dir, ".trash")
    os.makedirs(trash_dir, exist_ok=True)

    return MlflowClient()

def main():
    # Initialize MLflow with proper setup
    client = setup_mlflow()

    # Setup experiment with better error handling
    experiment_name = "documentation_generator"
    try:
        print(f"Attempting to create experiment: {experiment_name}")
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"Successfully created experiment with ID: {experiment_id}")
    except mlflow.exceptions.MlflowException as e:
        print(f"Could not create experiment: {str(e)}")
        try:
            experiment = mlflow.get_experiment_by_name(experiment_name)
            if experiment is None:
                raise Exception(f"Experiment {experiment_name} not found")
            experiment_id = experiment.experiment_id
            print(f"Found existing experiment with ID: {experiment_id}")
        except Exception as e:
            print(f"Error getting experiment: {str(e)}")
            raise

    # Verify experiment is set
    active_experiment = mlflow.get_experiment(experiment_id)
    print(f"Active experiment: {active_experiment.name} (ID: {active_experiment.experiment_id})")
    print(f"Artifact location: {active_experiment.artifact_location}")

    mlflow.set_experiment(experiment_name)

    # Training configuration
    training_args = {
        "output_dir": "documentation-assistant",
        "num_train_epochs": 3,
        "per_device_train_batch_size": 4,
        "learning_rate": 2e-4,
        "warmup_ratio": 0.05,
        "gradient_accumulation_steps": 4,
        "fp16": True,
        "logging_steps": 100,
        "save_strategy": "epoch",
    }

    # Execute ML pipeline with MLflow tracking
    model, tokenizer, model_setup_run = setup_model_and_tokenizer("google/gemma-2b")
    dataset_info, data_prep_run = load_and_prepare_dataset(num_examples=600)
    # Convert our dataset_info into a pytorch Dataset using this new function
    dataset = prepare_training_data(dataset_info,tokenizer)
    trainer, training_run = train_model(model, tokenizer, dataset, training_args)

    # Wait for model registration to complete
    import time
    time.sleep(10)  # Wait for 10 seconds (Adjust if necessary)

    # Get the latest model version
    # Get or create the registered model
    try:
        registered_model = client.get_registered_model(name=MODEL_NAME)
    except mlflow.exceptions.MlflowException:
        # If the model doesn't exist, create it
        registered_model = client.create_registered_model(name=MODEL_NAME)

    # Fetch the latest versions
    latest_versions = registered_model.latest_versions

    # Transition latest model version to production only if latest_versions is not empty
    if latest_versions:  # Check if latest_versions is not empty
        client.transition_model_version_stage(
            name=MODEL_NAME,
            version=latest_versions[0].version,
            stage="production"
        )
    else:
        print(f"No registered model versions found for {MODEL_NAME}. Skipping transition to production.")

    # Print registered models
    print("List of all registered models")
    print("=" * 80)
    # Use search_registered_models instead of list_registered_models
    [print(rm) for rm in client.search_registered_models()] # Changed line

    # Print specific model versions
    print(f"List of Model = {MODEL_NAME} and Versions")
    print("=" * 80)
    [print(mv) for mv in client.search_model_versions(f"name='{MODEL_NAME}'")]

    return {
        "model_setup_run": model_setup_run,
        "data_prep_run": data_prep_run,
        "training_run": training_run
    }

if __name__ == "__main__":
    main()

In [ ]:
!chmod 777 /content/mlruns.db  # Grants full permissions

In [ ]:
!ngrok authtoken "YOUR NGROK TOKEN"

In [ ]:
import mlflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Path to the MLflow model artifact
artifact_uri = "/content/model/model_state_dict.pth"  # Direct path to the artifact

# Download the artifact locally
local_path = mlflow.artifacts.download_artifacts(artifact_uri=artifact_uri)

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer and base model
base_model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)

# Load the fine-tuned state dictionary onto the GPU
state_dict = torch.load(local_path, map_location=device)

# Update the base model with the fine-tuned weights
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Test the model with an example input
test_input = "generate a simple python documentation"
inputs = tokenizer(test_input, return_tensors="pt").to(device)  # Move inputs to GPU

# Generate predictions
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)

# Decode the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


In [ ]:
import mlflow
import torch
import os

# Download the model artifact
local_path = mlflow.artifacts.download_artifacts(artifact_uri=model_uri)

# Load the model state dictionary
state_dict = torch.load(os.path.join(local_path, "model_state_dict.pth"))

# Load the model and tokenizer
model, tokenizer, _ = setup_model_and_tokenizer("google/gemma-2b")  # Assuming setup_model_and_tokenizer is defined

# Load the state dictionary into the model
model.load_state_dict(state_dict)

In [ ]:
mlflow.set_tracking_uri("/content/mlruns.db")

### Free up memory

In [ ]:
import torch

# Manually clear GPU memory
torch.cuda.empty_cache()  # Clears the cache
torch.cuda.reset_peak_memory_stats()  # Resets peak memory tracking


In [ ]:
torch.cuda.memory_allocated(), torch.cuda.memory_reserved()

In [ ]:
torch.cuda.empty_cache()